In [2]:
from phonecodes import phonecodes
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from timeit import default_timer as timer
from torch.nn import Transformer
from torch import Tensor
from sklearn.model_selection import train_test_split
import copy
import tqdm
import librosa
import seaborn as sns
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
import math
import os
import pandas as pd
import matplotlib.pyplot as plt
import textgrid
from scipy.spatial.distance import euclidean
import plotly.graph_objects as go
import pandas as pd
import jiwer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import nltk
import pickle
nltk.download('cmudict')
from nltk.corpus import cmudict
from transformers import AutoProcessor, AutoModelForCTC
from phonemizer.backend.espeak.wrapper import EspeakWrapper
import soundfile as sf
from sklearn.manifold import TSNE

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
processor_H = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
model_H = AutoModelForCTC.from_pretrained("facebook/hubert-large-ls960-ft")

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


c:\Users\Alex\anaconda3\envs\BayesPCN\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertForCTC: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForCTC 

In [3]:
os.environ['R_HOME'] = 'C:\\Program Files\\R\\R-4.4.1'
os.environ["PATH"] += os.pathsep + r"C:\Program Files\R\R-4.4.1\bin\x64"

from rpy2.robjects.packages import importr
from rpy2.robjects import Formula, pandas2ri

import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

In [4]:
def get_pathset(paths):
    return [os.path.join(dir, each_file) for dir, mid, files in os.walk(paths) for each_file in files if each_file.endswith(".wav")]
audio_dir =r"..\Feb09\audio_data"

In [5]:
NOISE_PATH = "../Feb_15/LTASmatched_noise.wav"
DEFAULT_SNR_LEVELS = [0, 0, 0, 0, 0, 0, 0, 0]
MAX_GAIN = 9

def get_group_id(index, group_size=15):
    return index // group_size


def safe_normalize(audio, max_peak=0.99):
    peak = np.max(np.abs(audio))
    if peak > max_peak:
        return audio * (max_peak / peak)
    return audio

def extend_noise(noise, target):
    while len(target)>len(noise):
        noise=np.append(noise,noise)
    return noise

def add_noise_v2(sentence_speech, sentence_index, 
                noise_path=NOISE_PATH, 
                snr_levels=DEFAULT_SNR_LEVELS,
                max_gain=MAX_GAIN):
    
    if len(sentence_speech) == 0:
        raise ValueError("Input speech cannot be empty")
    
    try:
        noise, sr = librosa.load(noise_path, sr=None)
        noise = librosa.resample(noise, orig_sr=sr, target_sr=16000)
    except Exception as e:
        raise RuntimeError(f"Failed to load noise file: {e}")
    
    group = get_group_id(sentence_index)
    if group >= len(snr_levels):
        return sentence_speech
    target_SNR = snr_levels[group]
    
    if target_SNR > max_gain:
        return sentence_speech
    
    speech_RMS = np.sqrt(np.mean(sentence_speech**2))
    noise_RMS = np.sqrt(np.mean(noise**2))
    scale_factor = (speech_RMS / (10 ** (target_SNR / 20))) / noise_RMS
    
    scaled_noise = noise * scale_factor
    extended_noise = extend_noise(scaled_noise, sentence_speech)
    mixed = sentence_speech + extended_noise[:len(sentence_speech)]

    mixed = safe_normalize(mixed)
    return np.clip(mixed, -1.0, 1.0)

def build_tSNE_matrices(audio_dir, model, processor):
    audio_path=get_pathset(audio_dir)
    sentences=[[] for i in range(60)]
    #print(len(sentences))
    for each_path in tqdm.tqdm(audio_path):
        audio, sr = librosa.load(each_path)
        wave_res = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        tg = textgrid.TextGrid.fromFile(each_path[:-3]+"TextGrid")
        tg_sentence = tg[0]
        for _,i in enumerate(tg[0]):
            if i.mark!="":
                tg_sentence[_-1].maxTime=tg_sentence[_].minTime
        tg_sentence = [i for i in tg_sentence if i.mark!=""]
        if "HT1" in each_path:
            set_list=get_sentence_ind(audio_path[0],human_result)
            #print("set hint1: ",len(set_list))
        else:
            set_list=get_sentence_ind(audio_path[1],human_result)
            #print("set hint2: ",len(set_list))
        tg_sentence=[tg_sentence[i] for i in set_list]
        #print("Now: ",len(tg_sentence))
        
        for _, each_sentence in enumerate(tg_sentence):
            
            start_sentence = int(each_sentence.minTime*16000)
            end_sentence = int(each_sentence.maxTime*16000)
            mixed_audio=add_noise_v2(wave_res[start_sentence:end_sentence], _)
            #wave_res[start_sentence:end_sentence]
            input=processor(mixed_audio, sampling_rate=16000, return_tensors="pt").input_values#.to(device)
            #model.to(device)
            model.eval() 
            #out_encoder = []
            conv_outputs = []
            x = input.clone()  
            for conv_layer in model_H.hubert.feature_extractor.conv_layers:
                x = conv_layer(x)
                conv_outputs.append(x.unsqueeze(0).transpose(2,1))
            #with torch.no_grad():
            #    outputs = model(input, output_hidden_states=True)
            #    all_layer_outputs = outputs.hidden_states
                #out_encoder=model.hubert.feature_extractor(input).transpose(2,1).cpu().numpy()
                #out_encoder=model.hubert(input)[0].cpu().numpy()
                #out_encoder=model_H.hubert.feature_extractor.conv_layers[4](input).unsqueeze(0).transpose(2,1).cpu().numpy()
                #out_encoder=model_H.hubert.feature_projection(model_H.hubert.feature_extractor(input).transpose(2,1)[0]).cpu().numpy()
            #out_encoder=all_layer_outputs[12].cpu().numpy()
            out_encoder=conv_outputs[0].detach().numpy()
            if "HT1" in each_path:
                sentences[_].append(out_encoder)
            else:
                sentences[_+31].append(out_encoder)
        torch.cuda.empty_cache()
    torch.cuda.empty_cache()
    flatten=np.array([x for i in sentences for j in i for x in j[0]])
    tsne = TSNE(n_components=3, random_state=42)
    reduced_data = tsne.fit_transform(flatten)
    mean = np.mean(reduced_data, axis=0)
    std = np.std(reduced_data, axis=0)
    reduced_data= (reduced_data - mean) / std
    count=0
    
    for _,i in enumerate(sentences):
        for __,j in enumerate(i):
            sentences[_][__]=reduced_data[count:count+j.shape[1]]
            count+=j.shape[1]
    return sentences
def get_sentence_ind(audio_path,human_result):
    tg = textgrid.TextGrid.fromFile(audio_path[:-3]+"TextGrid")
    tg_sentence = [i for i in tg[0] if i.mark!=""]
    hint1=[]
    for _,i in enumerate(tg_sentence):
        if i.mark in set(human_result['sentence_test']):
            hint1.append(_)
    return hint1

human_result_path=r"..\Feb09\BBP-2023-TestData1.xlsx"
human_result = pd.read_excel(human_result_path)
human_result=human_result[human_result["training_condition"]!="a_control"]
#sentence_matrix = build_tSNE_matrices(audio_dir, model_H, processor_H)

#with open("hubert_T24.pkl", "wb") as f:
#    pickle.dump(sentence_matrix, f)

In [6]:
with open("hubert_T24.pkl", "rb") as file:
    sentence_matrix1 = pickle.load(file)

In [7]:
from numba import njit
@njit
def weighted_minkowski(vec1, vec2,  tau, w=1):

    total = 0.0
    for m in range(len(vec1)):
        diff = w*abs(vec1[m] - vec2[m])
        total += (diff ** tau)
    return total**(1/tau)#np.sqrt(total)

@njit
def dtw_sim(seq1, seq2,  tau, k):
    n, m = len(seq1), len(seq2)
    dtw_matrix = np.full((n+1, m+1), np.inf)
    dtw_matrix[0, 0] = 0.0

    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = weighted_minkowski(seq1[i-1], seq2[j-1], tau)
            dtw_matrix[i, j] = cost + min(dtw_matrix[i-1, j],    # insertion
                                         dtw_matrix[i, j-1],    # deletion
                                         dtw_matrix[i-1, j-1])  # match
    return np.exp(-(dtw_matrix[n, m]/((n+m)/2))*k)# change to *k,


def create_distance_matrix(sentence_matrix,tau, k):
    distance_matirx=np.zeros((60,4,4))
    for _,i in enumerate(sentence_matrix):
        for _1,each_talker1 in enumerate(i):
            for _2,each_talker2 in enumerate(i):
                distance_matirx[_][_1][_2]=dtw_sim(each_talker1, each_talker2, tau, k)
    return distance_matirx
#distance_matirx=create_distance_matrix(sentence_matrix1,2,1)

In [ ]:


def sim_measure(df, distance_matirx,tg_sentence_list,talkers):
    out_df=pd.DataFrame(columns=['Condition', 'TrainingTalker', 'TestTalker','ListenerID' ,'Sentence',  'similarity', 'score_test_logit','numCorrect','numWord'])
    for each_ in tqdm.tqdm(df.values):
        #distance_list=[]
        
        
        if each_[1][0]=="b":
            ind_T = talkers.index(each_[1][-3:])
            arr = distance_matirx[tg_sentence_list.index(each_[df.columns.get_loc("sentence_test")])][ind_T]
            ind_test=talkers.index(each_[-3])
            out_df.loc[len(out_df)]=[each_[df.columns.get_loc("st_mt_cnt")],
                                        each_[df.columns.get_loc("training_condition")],
                                        each_[df.columns.get_loc("speaker_test")],
                                        str(each_[df.columns.get_loc("id2")]),
                                        each_[df.columns.get_loc("sentence_test")],
                                        arr[ind_test],
                                        each_[df.columns.get_loc("score_test_logit")],
                                        each_[df.columns.get_loc("score_test")],
                                        each_[df.columns.get_loc("possible_score_test")]
                                        ]
        else:
            ind_T = talkers.index(each_[1][-3:])
            ind_test = talkers.index(each_[-3])
            arr = distance_matirx[tg_sentence_list.index(each_[df.columns.get_loc("sentence_test")])][ind_T]
            #indices=np.nonzero(arr)
            arr_non_1 = arr[arr!=1]
            if each_[1][-3:] == each_[-3]:
                out_df.loc[len(out_df)]=[each_[df.columns.get_loc("st_mt_cnt")],
                                         each_[df.columns.get_loc("training_condition")],
                                         each_[df.columns.get_loc("speaker_test")],
                                         str(each_[df.columns.get_loc("id2")]),
                                         each_[df.columns.get_loc("sentence_test")],
                                         np.mean(arr_non_1),
                                         each_[df.columns.get_loc("score_test_logit")],
                                         each_[df.columns.get_loc("score_test")],
                                         each_[df.columns.get_loc("possible_score_test")]
                                         ]
    return out_df



In [86]:
human_result.values[0]

array(['55ea70c37480920010aa9982', 'b_SPA', 1.0, 5, 5,
       'SWIMMERS CAN HOLD THEIR BREATH', 'test', 'Test', 'SPA',
       2.397895272798371, 'st '], dtype=object)

In [ ]:
#tg_sentence_list.index(df.values[0][df.columns.get_loc("sentence_test")])
audio_path=get_pathset(audio_dir)
tg = textgrid.TextGrid.fromFile(audio_path[0][:-3]+"TextGrid")
tg_sentence = [i for i in tg[0] if i.mark!=""]
hint1_list=get_sentence_ind(audio_path[0],human_result)
tg_sentence=[tg_sentence[i].mark for i in hint1_list]
tg1 = textgrid.TextGrid.fromFile(audio_path[1][:-3]+"TextGrid")
tg_sentence1 = [i for i in tg1[0] if i.mark!=""]
hint2_list=get_sentence_ind(audio_path[1],human_result)
tg_sentence1=[tg_sentence1[i].mark for i in hint2_list]
tg_sentence_list=tg_sentence+tg_sentence1
tg_sentence_list.index(human_result.values[0][human_result.columns.get_loc("sentence_test")])

17

In [9]:
audio_path=get_pathset(audio_dir)
tg = textgrid.TextGrid.fromFile(audio_path[0][:-3]+"TextGrid")
tg_sentence = [i for i in tg[0] if i.mark!=""]
hint1_list=get_sentence_ind(audio_path[0],human_result)
tg_sentence=[tg_sentence[i].mark for i in hint1_list]
tg1 = textgrid.TextGrid.fromFile(audio_path[1][:-3]+"TextGrid")
tg_sentence1 = [i for i in tg1[0] if i.mark!=""]
hint2_list=get_sentence_ind(audio_path[1],human_result)
tg_sentence1=[tg_sentence1[i].mark for i in hint2_list]
tg_sentence_list=tg_sentence+tg_sentence1
tg_sentence_list.index(human_result.values[0][human_result.columns.get_loc("sentence_test")])

tg_sentence_list

#distance_matrix = create_distance_matrix(sentence_matrix1,tau)
#distance_matrix = np.exp(-1*np.array(distance_matrix)**k)

#out_df=sim_measure(human_result, distance_matrix, tg_sentence_list, talkers)
#out_df["numIncorrect"]=out_df["numWord"]-out_df["numCorrect"]
#out_df["prop_correct"]=out_df["numCorrect"]/out_df["numWord"]
#out_df.to_csv("distances_df_T_min_tau=1.csv", index=False)


['A BOY FELL FROM A WINDOW',
 'BIG DOGS CAN BE DANGEROUS',
 'THE SHOES WERE VERY DIRTY',
 'THE PLAYER LOST A SHOE',
 "SHE'S DRINKING FROM HER OWN CUP",
 'THE PICTURE CAME FROM A BOOK',
 'THE CAR IS GOING TOO FAST',
 'THE PAINT DRIPPED ON THE GROUND',
 'THE TOWEL FELL ON THE FLOOR',
 'THE KITCHEN WINDOW WAS CLEAN',
 'THE MAILMAN BROUGHT A LETTER',
 'THE MOTHER HEARD THE BABY',
 'SHE FOUND HER PURSE IN THE TRASH',
 "IT'S TIME TO GO TO BED",
 'MOTHER READ THE INSTRUCTIONS',
 'THE DOG IS EATING SOME MEAT',
 'THE PAINTER USES A BRUSH',
 'SWIMMERS CAN HOLD THEIR BREATH',
 "THEY'RE PUSHING AN OLD CAR",
 'THEY HAD TWO EMPTY BOTTLES',
 'THE DOG SLEEPS IN A BASKET',
 "THEY'RE PLAYING IN THE PARK",
 'THE BABY SLEPT ALL NIGHT',
 'THE SALT SHAKER IS EMPTY',
 'THE POLICEMAN KNOWS THE WAY',
 'THE BUCKETS FILL UP QUICKLY',
 'THE JANITOR SWEPT THE FLOOR',
 'THE LADY WASHED THE SHIRT',
 'THE MATCH BOXES ARE EMPTY',
 'THE MAN IS PAINTING A SIGN',
 'THE DOG CAME HOME AT LAST',
 'THEY HEARD A FUNNY NOISE',

In [10]:
audio_path=get_pathset(audio_dir)
talkers=[os.path.basename(i)[10:13] for i in audio_path if "HT1" in i]
talkers=['BRP', 'FAR', 'TUR', 'SPA']

In [12]:
human_result_path=r"..\Feb09\BBP-2023-TestData2.xlsx"
human_result = pd.read_excel(human_result_path)
human_result=human_result[human_result["training_condition"]!="a_control"]
human_result[human_result["sentence_test"]=='A BOY FELL FROM A WINDOW']

,id2,training_condition,prop_correct_test,score_test,possible_score_test,sentence_test,sessiontype_test,group,speaker_test,score_test_logit,st_mt_cnt
43,55ea70c37480920010aa9982,b_SPA,0.500000,3,6,A BOY FELL FROM A WINDOW,test,Test,TUR,0.000000,st
104,572bcf0d34b25a000edd2cf1,b_FAR,0.000000,0,6,A BOY FELL FROM A WINDOW,test,Test,TUR,-2.564949,st
177,57e91ff08dcd2c00014bea18,c_noTUR,0.500000,3,6,A BOY FELL FROM A WINDOW,test,Test,TUR,0.000000,mt
194,57fbd14e279ffd000124cc36,b_BRP,0.333333,2,6,A BOY FELL FROM A WINDOW,test,Test,TUR,-0.587787,st
289,59042666cd3f250001dcf8f9,c_noFAR,0.666667,4,6,A BOY FELL FROM A WINDOW,test,Test,TUR,0.587787,mt
...,...,...,...,...,...,...,...,...,...,...,...
9829,62029b1eef1cb18a1337c797,c_noTUR,0.333333,2,6,A BOY FELL FROM A WINDOW,test,Test,TUR,-0.587787,mt
9894,2003,b_BRP,0.000000,0,6,A BOY FELL FROM A WINDOW,test,Test,TUR,-2.564949,st
9911,2005,b_TUR,0.000000,0,6,A BOY FELL FROM A WINDOW,test,Test,TUR,-2.564949,st
10005,2015,b_TUR,0.000000,0,6,A BOY FELL FROM A WINDOW,test,Test,TUR,-2.564949,st


In [50]:
def objective_function(params, sentence_matrix1, human_result, tg_sentence_list, talkers):
    tau, k = params
    similarity_matrix = create_distance_matrix(sentence_matrix1, tau, k)
    #similarity_matrix = np.exp(-1*np.array(distance_matrix)*k)
    out_df=sim_measure(human_result, similarity_matrix,tg_sentence_list,talkers)
    out_df['numIncorrect'] = out_df['numWord'] - out_df['numCorrect']
    out_df['prop_correct'] = out_df['numCorrect'] / out_df['numWord']
    
    
    out_df['similarity_scaled'] = (out_df['similarity'] - np.mean(out_df['similarity'])) / (2 * np.std(out_df['similarity']))
    mask_mt = (out_df["Condition"] == "mt") & (out_df["TrainingTalker"].str[-3:] == out_df["TestTalker"])
    mask_st = (out_df["Condition"] == "st ") & (out_df["TrainingTalker"].str[-3:] != out_df["TestTalker"])
    new_df=out_df[mask_mt | mask_st]
    
    if np.std(new_df['similarity']) < 0.01:
        return {
                    "tau": [tau], 
                    "k": [k], 
                    "z-value": [0],  
                    "mean_sim": [float(np.mean(new_df["similarity"]))], 
                    "sd_sim": [float(np.std(new_df["similarity"]))],  
                    "error": ["std_sim<0.0001"]  # 
                }
    else:
        try:
            import rpy2.robjects as ro
            import rpy2
            from rpy2.robjects import pandas2ri
            
            pandas2ri.activate()
            base = importr('base')
            stats = importr('stats')
            lme4 = importr('lme4')
            ro.r('options(warn=2)')
            r_data = ro.conversion.py2rpy(new_df)
            #r_data = pandas2ri.py2rpy(new_df)
            formula = Formula('cbind(numCorrect, numIncorrect) ~ 1 + similarity_scaled + (1| TestTalker) + (1|Sentence)')
            glmerControl = lme4.glmerControl(optimizer="bobyqa", optCtrl=ro.vectors.ListVector({'maxfun': 1e6}))
        except Exception as e:
            return {
                    "tau": [tau], 
                    "k": [k], 
                    "z-value": [0],  
                    "mean_sim": [float(np.mean(new_df["similarity"]))], 
                    "sd_sim": [float(np.std(new_df["similarity"]))],  
                    "error": [e]  # 
                }
        try:
            model = lme4.glmer(formula, data=r_data, control=glmerControl, family=stats.binomial(link="logit"))
            #log_likelihood = ro.r['logLik'](model)
            summary = base.summary(model)
            coefficients = summary.rx2('coefficients')
            #z_value = coefficients.rx('similarity_scaled', 'z value')[0]
            z_value=coefficients[1][2]
            return {
                    "tau": [tau], 
                    "k": [k], 
                    "z-value": [z_value],  
                    "mean_sim": [float(np.mean(new_df["similarity"]))], 
                    "sd_sim": [float(np.std(new_df["similarity"]))],  
                    "error": ["NA"]  # 
                }
        except Exception as e:
            return {
                    "tau": [tau], 
                    "k": [k], 
                    "z-value": [0],  
                    "mean_sim": [float(np.mean(new_df["similarity"]))], 
                    "sd_sim": [float(np.std(new_df["similarity"]))],  
                    "error": [str(e)]  # 
                }
    

In [83]:
similarity_matrix = create_distance_matrix(sentence_matrix1, 1, 7)
out_df=sim_measure(human_result, similarity_matrix,tg_sentence_list,talkers)
out_df['numIncorrect'] = out_df['numWord'] - out_df['numCorrect']
out_df['prop_correct'] = out_df['numCorrect'] / out_df['numWord']


out_df['similarity_scaled'] = (out_df['similarity'] - np.mean(out_df['similarity'])) / (2 * np.std(out_df['similarity']))
mask_mt = (out_df["Condition"] == "mt") & (out_df["TrainingTalker"].str[-3:] == out_df["TestTalker"])
mask_st = (out_df["Condition"] == "st ") & (out_df["TrainingTalker"].str[-3:] != out_df["TestTalker"])
new_df=out_df[mask_mt | mask_st]

100%|██████████| 10080/10080 [00:04<00:00, 2487.05it/s]


In [84]:
pandas2ri.activate()
base = importr('base')
stats = importr('stats')
lme4 = importr('lme4')
ro.r('options(warn=2)')
r_data = ro.conversion.py2rpy(new_df)
#r_data = pandas2ri.py2rpy(new_df)
formula = Formula('cbind(numCorrect, numIncorrect) ~ 1 + similarity_scaled + (1| TestTalker) + (1|Sentence)')
glmerControl = lme4.glmerControl(optimizer="bobyqa", optCtrl=ro.vectors.ListVector({'maxfun': 1e6}))
model = lme4.glmer(formula, data=r_data, control=glmerControl, family=stats.binomial(link="logit"))

summary = base.summary(model)
coefficients = summary.rx2('coefficients')
#z_value = coefficients.rx('similarity_scaled', 'z value')[0]
z_value=coefficients[1][2]

R[write to console]: Error: (converted from warning) Some predictor variables are on very different scales: consider rescaling



RRuntimeError: Error: (converted from warning) Some predictor variables are on very different scales: consider rescaling


In [95]:
print(r_data[,"simiarity"])

SyntaxError: invalid syntax (1003209039.py, line 1)

In [19]:
tau_values = np.arange(-1,1,0.25)
tau_values = np.array([-1.  , -0.75, -0.5 , -0.25,  0.25,  0.5 ,  0.75])

In [96]:
tau_values = np.arange(-1,1,0.25)
tau_values = np.array([-1.  , -0.75, -0.5 , -0.25,  0.25,  0.5 ,  0.75, 1])
k_values = 10**np.arange(-4,2.2,0.2)
import itertools
import multiprocessing
import time
from joblib import Parallel, delayed
pandas2ri.activate()
params_grid=list(itertools.product(tau_values, k_values))



start_time = time.time()
n_jobs = min(31, multiprocessing.cpu_count())
results = Parallel(n_jobs=n_jobs)(delayed(objective_function)(_, sentence_matrix1, human_result, tg_sentence_list, talkers) for _ in params_grid)
end_time = time.time()


In [81]:
results

[{'tau': [1],
  'k': [0.0001],
  'z-value': [0],
  'mean_sim': [0.9998171914807771],
  'sd_sim': [2.921271850786103e-05],
  'error': ['std_sim<0.02']},
 {'tau': [1],
  'k': [0.00012589254117941674],
  'z-value': [0],
  'mean_sim': [0.999769863600878],
  'sd_sim': [3.677524350412713e-05],
  'error': ['std_sim<0.02']},
 {'tau': [1],
  'k': [0.00015848931924611142],
  'z-value': [0],
  'mean_sim': [0.999710284775556],
  'sd_sim': [4.629508542093144e-05],
  'error': ['std_sim<0.02']},
 {'tau': [1],
  'k': [0.0001995262314968881],
  'z-value': [0],
  'mean_sim': [0.9996352849385336],
  'sd_sim': [5.827856796639259e-05],
  'error': ['std_sim<0.02']},
 {'tau': [1],
  'k': [0.0002511886431509582],
  'z-value': [0],
  'mean_sim': [0.999540874390903],
  'sd_sim': [7.336283699890223e-05],
  'error': ['std_sim<0.02']},
 {'tau': [1],
  'k': [0.00031622776601683826],
  'z-value': [0],
  'mean_sim': [0.9994220322661076],
  'sd_sim': [9.234957127304789e-05],
  'error': ['std_sim<0.02']},
 {'tau': [1],

In [24]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.interpolate import griddata

def plot_gridsearch_result(results):
    df = pd.json_normalize(results)
    df['tau'] = df['tau'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['k'] = df['k'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['z-value'] = df['z-value'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['mean_sim'] = df['mean_sim'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['sd_sim'] = df['sd_sim'].apply(lambda x: x[0] if isinstance(x, list) else x)
    tau_vals = tau_values  
    k_vals = k_values
    tau_grid, k_grid = np.meshgrid(tau_vals, k_vals)  



    fig = make_subplots(
        rows=1, cols=3, 
        subplot_titles=['z-value', 'mean_sim', 'sd_sim'], 
        specs=[[{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}]]
    )


    z_grid = griddata((df['tau'], df['k']), df['z-value'], (tau_grid, k_grid), method='nearest')
    mean_sim_grid = griddata((df['tau'], df['k']), df['mean_sim'], (tau_grid, k_grid), method='nearest')
    sd_sim_grid = griddata((df['tau'], df['k']), df['sd_sim'], (tau_grid, k_grid), method='nearest')


    fig.add_trace(go.Surface(z=z_grid, x=tau_grid, y=k_grid, colorscale='RdYlGn', showscale=False), row=1, col=1)
    fig.add_trace(go.Surface(z=mean_sim_grid, x=tau_grid, y=k_grid, colorscale='RdYlGn', showscale=False), row=1, col=2)
    fig.add_trace(go.Surface(z=sd_sim_grid, x=tau_grid, y=k_grid, colorscale='RdYlGn', showscale=False), row=1, col=3)


    fig.update_layout(
        width=1700, 
        height=600,  
        title='3D Surface Plots for z-value, mean_sim, and sd_sim from LSR of 24th-Transformer-layer (Bradlow 2023)',
        scene=dict(
            xaxis_title='Tau',
            yaxis_title='K',
            zaxis_title='Z-Value',
            yaxis=dict(
                type='log',
                tickvals=[1e-4, 1e-3, 1e-2, 1e-1, 1, 10], 
                ticktext=['1e-4', '1e-3', '1e-2', '1e-1', '1', '10'],
            )
        ),
        scene2=dict(
            xaxis_title='Tau',
            yaxis_title='K',
            zaxis_title='Mean-Sim',
            yaxis=dict(
                type='log',
                tickvals=[1e-4, 1e-3, 1e-2, 1e-1, 1, 10], 
                ticktext=['1e-4', '1e-3', '1e-2', '1e-1', '1', '10'],
            )
        ),
        scene3=dict(
            xaxis_title='Tau',
            yaxis_title='K',
            zaxis_title='SD-Sim',
            yaxis=dict(
                type='log',
                tickvals=[1e-4, 1e-3, 1e-2, 1e-1, 1, 10], 
                ticktext=['1e-4', '1e-3', '1e-2', '1e-1', '1', '10'],
            ),
            zaxis=dict(
                range=[0,0.5]
            )
        ),
        showlegend=False,
        coloraxis_showscale=False
        
    )
    fig.show()


In [97]:
plot_gridsearch_result(results)